# S-matrix theory

In [1]:
import numpy as np
import matplotlib as plt
import sympy as sp

from logzero import logger
from IPython.display import display, Math

%matplotlib widget

# Composing scattering at interfaces

In [2]:
x1, x2, x3, x4, x5, x6 = sp.symbols( ' '.join( [ 'x_%d'%i for i in range( 1, 7 ) ] ) )
T12, T21, R12, R21 = sp.symbols( ' '.join( [ f'{lbl}{n%2+1}{(n+1)%2+1}' for lbl in [ 'T', 'R' ] for n in [ 0, 1 ] ] ) ) 
T23, T32, R23, R32 = sp.symbols( ' '.join( [ f'{lbl}{n%2+2}{(n+1)%2+2}' for lbl in [ 'T', 'R' ] for n in [ 0, 1 ] ] ) ) 
k, L = sp.symbols( 'k L' )

In [3]:
# intermediate solution
eq1 = T12*x1 + R21*x5 - x6
eq2 = R12*x1 + T21*x5 - x4
intermediate_solution = sp.linsolve( [ eq1, eq2 ], ( x5, x6 ) )
x5, x6 = list( intermediate_solution )[0]

In [4]:
x5

(-R12*x_1 + x_4)/T21

In [5]:
sp.simplify( x6 )

(-R12*R21*x_1 + R21*x_4 + T12*T21*x_1)/T21

In [6]:
# final solution
w = sp.exp( sp.I*k*L )
wconj = sp.exp( -sp.I*k*L)
eq3 = T23*x6*w + R32*x2 - x3
eq4 = R23*x6*w + T32*x2 - x5*wconj
final_solution  = sp.linsolve( [ eq3, eq4 ], ( x3, x4 ) )
x3, x4 = list( final_solution )[0] 

logger.info( 'Final solution: ' )
for exp, lbl in zip( [ x3, x4 ], [ 'x_3', 'x_4' ] ):
    display( Math( f'\\Large{sp.latex( sp.simplify( exp ) )}' ) )

eq5 = list( final_solution )[0][0] - sp.symbols( 'x3' )
eq6 = list( final_solution )[0][1] - sp.symbols( 'x4' )
S, coeff = sp.linear_eq_to_matrix( [ eq5, eq6 ], ( x1, x2 ) )

[I 250923 11:58:14 1315166222:9] Final solution: 


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
logger.info( 'Final S-matrix: ' )
Sl = sp.latex(sp.simplify(S))
display( Math( f'\\Large{sp.latex(sp.simplify(S))}' ) )

[I 250923 11:58:14 283930408:1] Final S-matrix: 


<IPython.core.display.Math object>

# Does the Redheffer product support this?

In [8]:
def smatrix( A11, A12, A21, A22, k, L ):
    '''
    S-matrix for a single interface with given reflection/transmission coefficients, 
    located L (1-D) and intervening medium corresponding to a wave vector k. 
    '''
    w = sp.exp( sp.I * k * L )
    return sp.Matrix( 
        [ 
            [ A11*w, A12 ], 
            [ A21*w*w, A22*w ]
        ]
    )

def redheffer( A, B ):
    denom = 1 - A[0,1]*B[1,0]
    Mred = sp.Matrix( 
        [ 
            [ A[0,0]*B[0,0]/denom, B[0,1] + ( B[0,0]*B[1,1]*A[0,1] ) / denom ], 
            [ A[1,0] + ( A[0,0]*A[1,1]*B[1,0] ) / denom, A[1,1]*B[1,1]/denom ]
        ]
    )
    return Mred

In [9]:
M0 = smatrix( T12, R21, R12, T21, k, 0 )
M1 = smatrix( T23, R32, R23, T32, k, L )
Mp = redheffer( M0, M1 )

In [10]:
M0

Matrix([
[T12, R21],
[R12, T21]])

In [11]:
M1

Matrix([
[  T23*exp(I*L*k),            R32],
[R23*exp(2*I*L*k), T32*exp(I*L*k)]])

In [12]:
sp.simplify( Mp )

Matrix([
[                                        -T12*T23*exp(I*L*k)/(R21*R23*exp(2*I*L*k) - 1), (-R21*T23*T32*exp(2*I*L*k) + R32*(R21*R23*exp(2*I*L*k) - 1))/(R21*R23*exp(2*I*L*k) - 1)],
[(R12*(R21*R23*exp(2*I*L*k) - 1) - R23*T12*T21*exp(2*I*L*k))/(R21*R23*exp(2*I*L*k) - 1),                                          -T21*T32*exp(I*L*k)/(R21*R23*exp(2*I*L*k) - 1)]])

In [13]:
sp.simplify( S )

Matrix([
[                                        -T12*T23*exp(I*L*k)/(R21*R23*exp(2*I*L*k) - 1), (R21*R23*R32*exp(2*I*L*k) - R21*T23*T32*exp(2*I*L*k) - R32)/(R21*R23*exp(2*I*L*k) - 1)],
[(R12*R21*R23*exp(2*I*L*k) - R12 - R23*T12*T21*exp(2*I*L*k))/(R21*R23*exp(2*I*L*k) - 1),                                         -T21*T32*exp(I*L*k)/(R21*R23*exp(2*I*L*k) - 1)]])

In [14]:
sp.simplify( Mp - S )

Matrix([
[0, 0],
[0, 0]])